In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import nltk
import pandas as pd
from nltk.util import ngrams
import pickle
import torch
from torch.autograd import Variable
import sys

In [ ]:
!pip install lime

In [ ]:
!pip install skipthoughts

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
my_dict = pickle.load(open('gdrive/MyDrive/CS626/Project/Data/my_dict.pkl', 'rb'))
biskip = pickle.load(open('gdrive/MyDrive/CS626/Project/Data/biskip.pkl', 'rb'))

In [ ]:
def neural_features(headlines,bodies):
  MAX_HEAD = max(len(ele) for ele in headlines)
  MAX_BODY = max(len(ele) for ele in bodies)
  headlines_to_ids = np.zeros((len(headlines),MAX_HEAD+1))
  bodies_to_ids = np.zeros((len(bodies),MAX_BODY+1))
  headlines_encodings = np.zeros((len(headlines),2400))
  bodies_encodings = np.zeros((len(bodies),2400))
  for i in range(len(headlines)):
    headline = headlines[i].split()
    headline.append('<eos>')
    body = bodies[i].split()
    body.append('<eos>')
    j=0
    for word in headline:
      try:
        headlines_to_ids[i][j] = my_dict[word]
      except KeyError:
        pass
      j+=1
    j=0
    for word in body:
      try:
        bodies_to_ids[i][j] = my_dict[word]
      except KeyError:
        pass
      j+=1
  last_temp = len(headlines) - len(headlines)%50
  for i in range(0,len(headlines),50):
    # print(i)
    input1 = Variable(torch.LongTensor(headlines_to_ids[i:i+50]))
    input2 = Variable(torch.LongTensor(bodies_to_ids[i:i+50]))
    headline_output = biskip(input1).detach().numpy()
    body_output = biskip(input2).detach().numpy()
    headlines_encodings[i:i+50] = headline_output[0:50]
    bodies_encodings[i:i+50] = body_output[0:50]
  if(last_temp != len(headlines)):
    input1 = Variable(torch.LongTensor(headlines_to_ids[last_temp:]))
    input2 = Variable(torch.LongTensor(bodies_to_ids[last_temp:]))
    headline_output = biskip(input1).detach().numpy()
    body_output = biskip(input2).detach().numpy()
    headlines_encodings[last_temp:] = headline_output[:]
    bodies_encodings[last_temp:] = body_output[:]

  feat1 = np.zeros((len(headlines),2400))
  feat2 = np.zeros((len(headlines),2400))
  i = 0
  for h_vector,b_vector in zip(headlines_encodings,bodies_encodings):
    feat1[i] = np.multiply(h_vector,b_vector)
    feat2[i] = np.absolute(h_vector-b_vector)
    i+=1

  final_neural_features = np.concatenate((feat1,feat2),axis = 1)
  return final_neural_features


In [ ]:
head_demo_sent1 = 'ISIL Beheads American Photojournalist in Iraq' #discuss
body_demo_sent1 = 'James Foley, an American journalist who went missing in Syria more than a year ago, has reportedly been executed by the Islamic State, a militant group formerly known as ISIS.Video and photos purportedly of Foley emerged on Tuesday. A YouTube video -- entitled "A Message to #America (from the #IslamicState)" -- identified a man on his knees as "James Wright Foley," and showed his execution.This is a developing story. Check back here for updates.'
head_demo_sent2 = 'A Russian Guy Says His Justin Bieber Ringtone Saved Him From A Bear Attack' #unrelated
body_demo_sent2 = 'A bereaved Afghan mother took revenge on the Taliban after watching them kill her son in an ambush. Reza Gul killed 25 Taliban fighters and injured five others in a seven-hour gunbattle in Farah province.Gul, who was joined by her daughter and daughter in-law, engaged the Taliban using AK-47s and grenades, despitenever before having used a weapon.The embattled mother told Tolo news, a 24-hour Afghan news broadcaster, she was awakened by shots early Tuesday. After seeing that her son had been killed, Gul and the other two women fought back.“I couldn stop myself and picked up a weapon,” Gul told Tolo News. “I went to the check post and began shooting back.”Seema, her daughter-in-law, added: “The fighting was intensified when we reached the battlefield along with light and heavy weapons. We were committed to fight until the last bullet.”Gul said that the battlefield was covered in Talib fighters after the deadly exchange ended.While the Taliban have not publicly commented on the incident, the Afghan government labeled it a symbol of a public uprising against the Taliban.Taliban and other groups have regained large swathes of the country as U.S. and NATO forces slowly pull out troops after 14 years of war. The Taliban have targeted government and foreign infrastructure as the group attempts to claw back power it lost in 2001.While the Taliban have made key gains in rural regions, members continue to employ suicide bomber tactics in well protected towns and cities. Earlier this week, 50 people were killed after a suicide bomber detonated a vest during a volleyball competition in Yahyakahil, Paktika province.That particular attack prompted President Ashraf Ghani to order a complete overview of the country’s defense forces and to rethink the ban on nighttime raids, which were outlawed by his predecessor, Hamid Karzai.'

In [ ]:
headlines_org = []
bodies_org = []
headlines_org.append(head_demo_sent1)
bodies_org.append(body_demo_sent1)
headlines_org.append(head_demo_sent2)
bodies_org.append(body_demo_sent2)
n_feats = neural_features(headlines_org,bodies_org)

In [ ]:
print(n_feats.shape)

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

In [ ]:

_wnl = nltk.WordNetLemmatizer()


def normalize_word(w):
    return _wnl.lemmatize(w).lower()


def get_tokenized_lemmas(s):
    return [normalize_word(t) for t in nltk.word_tokenize(s)]


def clean(s):
    # Cleans a string: Lowercasing, trimming, removing non-alphanumeric

    return " ".join(re.findall(r'\w+', s, flags=re.UNICODE)).lower()


def remove_stopwords(l):
    # Removes stopwords from a list of tokens
    return [w for w in l if w not in feature_extraction.text.ENGLISH_STOP_WORDS]

def preprocess(headlines,bodies):
  n_headlines, n_bodies =[],[]
  for i, (headline, body) in enumerate(zip(headlines, bodies)):
    clean_headline = clean(headline)
    clean_body = clean(body)
    clean_headline = get_tokenized_lemmas(clean_headline)
    clean_body = get_tokenized_lemmas(clean_body)
    clean_headline = remove_stopwords(clean_headline)
    clean_body = remove_stopwords(clean_body)
    n_headlines.append(headline)
    n_bodies.append(body)
  n_headlines_df=pd.DataFrame(n_headlines,columns=['Headline'])
  n_bodies_df=pd.DataFrame(n_bodies,columns=['Body'])
  return n_headlines_df['Headline'], n_bodies_df['Body']


In [ ]:
df = pd.read_csv('gdrive/MyDrive/CS626/Project/Data/train_Set.csv')
def statistical_features(headlines,bodies):
  stop_words_l=stopwords.words('english')
  org_hs, org_bs = df['Headline'], df['Body']
  org_hs,org_bs = preprocess(org_hs,org_bs)
  headline_vectorizer = TfidfVectorizer()
  h1 = headline_vectorizer.fit(org_hs)
  h = h1.transform(headlines)
  body_vectorizer = TfidfVectorizer(max_features=10000-h.shape[1])
  b1 = body_vectorizer.fit(org_bs)
  b = b1.transform(bodies)
  statistical_features = np.concatenate((np.array(h.toarray()),np.array(b.toarray())),axis = 1)
  return statistical_features

In [ ]:
s_feats = statistical_features(headlines_org,bodies_org)

In [ ]:
print(s_feats.shape)

In [ ]:
!pip install vaderSentiment

In [ ]:
from nltk.util import ngrams
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
def external_features(headline,body):  
  eng_ext = []
  i = 0
  for sent1,sent2 in zip(headline,body):
    i+=1
    vec = []

    #character ngrams
    for n in range(2,17):
      n_grams_1 = ngrams(sent1.lower(), n)
      n_grams_2 = ngrams(sent2.lower(),n)
      vec.append(len(list(set(n_grams_1).intersection(set(n_grams_2)))))
      temp_c1=0
      temp_c2=0
      n_grams_1 = ngrams(sent1.lower(), n)
      n_grams_3 = ngrams(sent2.lower()[:255],n)
      temp_c1 = len(list(set(n_grams_1).intersection(set(n_grams_3))))
      n_grams_1 = ngrams(sent1.lower(), n)
      n_grams_4 = ngrams(sent2.lower()[:100],n)
      temp_c2 = len(list(set(n_grams_1).intersection(set(n_grams_4))))
      vec.append(temp_c1)
      vec.append(temp_c2)

    #word ngrams
    for n in range(2,7):
      n_grams_1 = ngrams(sent1.lower().split(), n)
      n_grams_2 = ngrams(sent2.lower().split(),n)
      vec.append(len(list(set(n_grams_1).intersection(set(n_grams_2)))))
      temp_c=0
      n_grams_1 = ngrams(sent1.lower().split(), n)
      n_grams_3 = ngrams(sent2.lower()[:255].split(),n)
      temp_c=len(list(set(n_grams_1).intersection(set(n_grams_3))))
      vec.append(temp_c)

    #no of common words between headline and body with respect to total words
    s1 = sent1.split()
    s2 = sent2.split()
    vec.append(len(set(s1).intersection(s2)) / float(len(set(s1).union(s2))))

    sid_obj = SentimentIntensityAnalyzer()
    d1 = sid_obj.polarity_scores(sent1)
    d2 = sid_obj.polarity_scores(sent2)
    vec.append(np.absolute(d1['neg']-d2['neg']))
    vec.append(np.absolute(d1['neu']-d2['neu']))
    vec.append(np.absolute(d1['pos']-d2['pos']))
    vec.append(np.absolute(d1['compound']-d2['compound']))

    eng_ext.append(vec)

  eng_ext = np.array(eng_ext)
  return eng_ext


In [ ]:
e_feats = external_features(headlines_org,bodies_org)

In [ ]:
print(e_feats.shape)

In [ ]:
model = pickle.load(open('gdrive/MyDrive/CS626/Project/Data/final_model.pkl', 'rb'))

In [ ]:
y_pred = model.predict(x=[n_feats,e_feats,s_feats])

In [ ]:
y_pred_ = np.argmax(y_pred,axis=1)

In [ ]:
print(y_pred_)

In [ ]:
for i in range(2):
  if(y_pred_[i] == 0):
      print("Headline and body pairs agree with each other")
  elif(y_pred_[i] == 1):
      print("Headline and body pairs disagree with each other")
  elif(y_pred_[i]==2):
      print("Headline and body pairs are discussion statements")
  elif(y_pred_[i]==3):
      print("Headline and body pairs are unrelated")

In [ ]:
from lime.lime_text import LimeTextExplainer
class_names=['agree','disagree','discussion','unrelated']
explainer= LimeTextExplainer(class_names=class_names)

In [ ]:
def predict_proba(arr):
  processed = []
  final_res = np.zeros((len(arr),4))
  i_ =0
  final_headlines = []
  final_bodies = []
  for i in arr:
    a = i.split('\n',1)
    headline = a[0]
    body = a[1]
    final_headlines.append(headline)
    final_bodies.append(body)
  res = model.predict(x=[neural_features(final_headlines,final_bodies),external_features(final_headlines,final_bodies),statistical_features(final_headlines,final_bodies)])
  return res

In [ ]:
exp = explainer.explain_instance(head_demo_sent1+'\n'+body_demo_sent1,predict_proba,num_features=20,top_labels=4,num_samples=5000)

In [ ]:
exp.show_in_notebook(text=True,labels=(2,))

In [ ]:
exp2 = explainer.explain_instance(head_demo_sent2+'\n'+body_demo_sent2,predict_proba,num_features=20,top_labels=4,num_samples=5000)

In [ ]:
exp2.show_in_notebook(text=True,labels=(3,))